# AIFFEL Hackerthon 1 - Visitor prediction

[colab](https://colab.research.google.com/drive/1smPns4wZvFKdrd9qsm7YYqI7nasYJn73?usp=sharing)

## 0. Kaggle에서 데이터 다운로드 받기

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
ls -1ha kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

In [ ]:
!kaggle competitions download -c recruit-restaurant-visitor-forecasting

## 1. 모듈 임포트, 데이터 준비하기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
air_reserve = pd.read_csv("air_reserve.csv.zip")
hpg_reserve = pd.read_csv("hpg_reserve.csv.zip")
air_store_info = pd.read_csv("air_store_info.csv.zip")
hpg_store_info = pd.read_csv("hpg_store_info.csv.zip")
air_visit_data = pd.read_csv("air_visit_data.csv.zip")  
store_id_relation = pd.read_csv("store_id_relation.csv.zip")
date_info = pd.read_csv("date_info.csv.zip")
sample_submission = pd.read_csv("sample_submission.csv.zip")

In [ ]:
### 원본 저장하기 ###
air_reserve_orig = air_reserve.copy()
hpg_reserve_orig = hpg_reserve.copy()
air_store_info_orig = air_store_info.copy()
hpg_store_info_orig = hpg_store_info.copy()
air_visit_data_orig = air_visit_data.copy() ## EDA
store_id_relation_orig = store_id_relation.copy() ## mapping -> merge
date_info_orig = date_info.copy() ## holiday => reserve
sample_submission_orig = sample_submission.copy()

## 2. EDA

`air_reserve` 부터 `date_info`까지 하나씩 EDA를 해보겠습니다. (`sample_submission`의 경우 제출용 파일임으로 생략하겠습니다.)

### 2-1. Analysis of `air_reserve`
주의할 것은 `air_reserve`는 예약을 하고 방문한 사람의 데이터만 가지고 있습니다. 따라서 `air_visit_data`를 고려해보면, 같은 매장에 예약을 하지않고 방문하는 인원도 있을 수 있고, 또 예약을 애초에 받지 않는 매장도 있을 수 있습니다. 그리고 예약을 하고 온 인원도 포함되있을 것 같습니다. 이 내용은 아래에 별도로 정리를 해두었습니다.

따라서 `air_reserve` 데이터는 __예약한 인원 수__에 대한 데이터로 보는 것이 좋습니다. 실제 방문객에 대한 데이터로 사용하기는 어려울 수 있다는 의미입니다.

_reserve_visitors - the number of visitors for that reservation_

In [ ]:
air_reserve.head()

In [ ]:
air_reserve.describe()

In [ ]:
air_reserve.isnull().sum()

In [ ]:
air_reserve.dtypes

예약시간, 방문시간과 방문객 사이의 관계를 한번 알아보겠습니다.

우선 visit_datetime과 reserve_datetime에 해당하는 컬럼의 자료형을 object에서 datetime 으로 바꿔주겠습니다.

In [ ]:
air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])

그래프를 그리기 위해 날짜별로 인원수 합을 구해보겠습니다.

In [ ]:
visit_time_visitors = pd.pivot_table(air_reserve, index='visit_datetime', values='reserve_visitors', aggfunc='sum').reset_index()
reserve_time_visitors = pd.pivot_table(air_reserve, index='reserve_datetime', values='reserve_visitors', aggfunc='sum').reset_index()

print(f"visit_time_visitors has {len(visit_time_visitors)} rows.")
print(f"reserve_time_visitors has {len(reserve_time_visitors)} rows.")

display(visit_time_visitors.head())
display(reserve_time_visitors.head())

그래프를 그리려고 보니 각각 4975, 7513개의 그래프를 모두 막대그래프로 표현하는데는 무리가 있을 것 같습니다. 우선 25개 정도의 row를 대상으로 그래프를 그려보겠습니다.

In [ ]:
import matplotlib.dates as mdates
fig, ax = plt.subplots()
sns.barplot(x=visit_time_visitors["visit_datetime"].iloc[0:25], y=visit_time_visitors["reserve_visitors"].iloc[0:25], ax=ax)
x_tick_labels = [t.get_text()[:19] for t in ax.get_xticklabels()]
ax.set_xticklabels(x_tick_labels, rotation=90)


위와 같은 결과가 나옵니다. 2016년 1월 2일 17시가 가장 예약객이 많은 시간인 건 알 수 있지만, 그다지 유믜미한 정보를 찾을 수 있는 시각화는 아닙니다. 따라서 다음의 세부적인 경우로 나누어 데이터를 살펴보겠습니다.

#### 예약객 정보


1. 요일별 예약객 정보: 주말이나 공휴일
2. 월별 예약객 정보: 계절이랑 비슷할 것
3. 계절별 예약객 정보: 제철음식, 음식장르와 관련이 있을 것 같음
4. 시간대별: 저녁시간, 점심시간, ...

##### 1. 요일별 예약객 정보

In [ ]:
visit_datetime_day = air_reserve['visit_datetime'].dt.day_name()
reserve_datetime_day = air_reserve['reserve_datetime'].dt.day_name()

In [ ]:
air_reserve['visit_datetime_day'] = visit_datetime_day
air_reserve['reserve_datetime_day'] = reserve_datetime_day

In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
visit_day_visitors = pd.pivot_table(air_reserve, index='visit_datetime_day', values='reserve_visitors', aggfunc='sum').reindex(days).reset_index()
reserve_day_visitors = pd.pivot_table(air_reserve, index='reserve_datetime_day', values='reserve_visitors', aggfunc='sum').reindex(days).reset_index()

print(f"visit_day_visitors has {len(visit_day_visitors)} rows.")
print(f"reserve_day_visitors has {len(reserve_day_visitors)} rows.")


display(visit_day_visitors)
display(reserve_day_visitors)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_day_visitors, x="visit_datetime_day", y="reserve_visitors", ax=axs[0], palette=colors)

axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit day / Visitors")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height+1000), ha = 'center', color = 'k', style='italic')

axs[0].patches[4].set_linewidth('2')
axs[0].patches[4].set_edgecolor('darkorange')
axs[0].patches[5].set_linewidth('2')
axs[0].patches[5].set_edgecolor('darkorange')

axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

sns.barplot(data=reserve_day_visitors, x="reserve_datetime_day", y="reserve_visitors", ax=axs[1], palette=colors)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve day / Visitors")
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height+1000), ha = 'center', color = 'k', style='italic')

axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

위에서 볼 수 있듯이 예약하는 날짜와 방문객은 별로 상관이 없어 보입니다. 반면에 방문하는 날짜는 일반적으로 금요일, 토요일에 많이 올려있는 모습을 볼 수 있습니다.

##### 2. 월별 예약객 정보

In [ ]:
visit_datetime_month = air_reserve['visit_datetime'].dt.month_name()
reserve_datetime_month = air_reserve['reserve_datetime'].dt.month_name()
air_reserve['visit_datetime_month'] = visit_datetime_month
air_reserve['reserve_datetime_month'] = reserve_datetime_month

In [ ]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
visit_month_visitors = pd.pivot_table(air_reserve, index='visit_datetime_month', values='reserve_visitors', aggfunc='sum').reindex(months).reset_index()
reserve_month_visitors = pd.pivot_table(air_reserve, index='reserve_datetime_month', values='reserve_visitors', aggfunc='sum').reindex(months).reset_index()

print(f"visit_month_visitors has {len(visit_month_visitors)} rows.")
print(f"reserve_month_visitors has {len(reserve_month_visitors)} rows.")


display(visit_month_visitors)
display(reserve_month_visitors)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_month_visitors, x="visit_datetime_month", y="reserve_visitors", ax=axs[0], palette=colors, zorder=2)
axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit Month / Visitors")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')

axs[0].axhline(y = 13500, color = 'dodgerblue', ls = '--', zorder = 1)
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)


sns.barplot(data=reserve_month_visitors, x="reserve_datetime_month", y="reserve_visitors", ax=axs[1], palette=colors, zorder=2)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve Month / Visitors")
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')

axs[1].axhline(y = 20200, color = 'dodgerblue', ls = '--', zorder = 1)
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

월별 예약객/방문객을 구별해보면 유달리 5월부터 9월 사이에 상당히 적은 인원이 예약/방문을 하고 있으며 10월에도 비교적 적은 수를 보이고 있습니다. 무슨 이유가 있는 지 확실히 알 수는 없지만 air 서비스를 사용할 수 없었다거나 무슨 이유가 있을 것 같습니다.

##### 3. 계절별 예약 방문객 정보

`np.where`을 사용해 계절을 표현하는 컬럼을 만들어줍니다.

In [ ]:
air_reserve['visit_datetime_season'] = ""
air_reserve['reserve_datetime_season'] = ""

In [ ]:
air_reserve['visit_datetime_season'] = np.where(air_reserve['visit_datetime_month'].isin(["March", "April", "May"]), 'Spring', air_reserve['visit_datetime_season'])
air_reserve['visit_datetime_season'] = np.where(air_reserve['visit_datetime_month'].isin(["June", "July", "August"]), 'Summer', air_reserve['visit_datetime_season'])
air_reserve['visit_datetime_season'] = np.where(air_reserve['visit_datetime_month'].isin(["September", "October", "November"]), 'Fall', air_reserve['visit_datetime_season'])
air_reserve['visit_datetime_season'] = np.where(air_reserve['visit_datetime_month'].isin(["December", "January", "February"]), 'Winter', air_reserve['visit_datetime_season'])

In [ ]:
air_reserve['reserve_datetime_season'] = np.where(air_reserve['reserve_datetime_month'].isin(["March", "April", "May"]), 'Spring', air_reserve['reserve_datetime_season'])
air_reserve['reserve_datetime_season'] = np.where(air_reserve['reserve_datetime_month'].isin(["June", "July", "August"]), 'Summer', air_reserve['reserve_datetime_season'])
air_reserve['reserve_datetime_season'] = np.where(air_reserve['reserve_datetime_month'].isin(["September", "October", "November"]), 'Fall', air_reserve['reserve_datetime_season'])
air_reserve['reserve_datetime_season'] = np.where(air_reserve['reserve_datetime_month'].isin(["December", "January", "February"]), 'Winter', air_reserve['reserve_datetime_season'])

In [ ]:
seasons = ['Spring', 'Summer', 'Fall', 'Winter']
visit_season_visitors = pd.pivot_table(air_reserve, index='visit_datetime_season', values='reserve_visitors', aggfunc='sum').reindex(seasons).reset_index()
reserve_season_visitors = pd.pivot_table(air_reserve, index='reserve_datetime_season', values='reserve_visitors', aggfunc='sum').reindex(seasons).reset_index()

print(f"visit_season_visitors has {len(visit_season_visitors)} rows.")
print(f"reserve_season_visitors has {len(reserve_season_visitors)} rows.")


display(visit_season_visitors)
display(reserve_season_visitors)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_season_visitors, x="visit_datetime_season", y="reserve_visitors", ax=axs[0], palette=colors, zorder=2)
axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit Month / Visitors")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')

axs[0].axhline(y = 9200, color = 'dodgerblue', ls = '--', zorder = 1)
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)


sns.barplot(data=reserve_season_visitors, x="reserve_datetime_season", y="reserve_visitors", ax=axs[1], palette=colors, zorder=2)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve Month / Visitors")
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')

axs[1].axhline(y = 11000, color = 'dodgerblue', ls = '--', zorder = 1)
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

월별 정보와 비슷한 모습을 보입니다. 전체적으로 합쳐져서 여름에는 다른 계절에 비해 확연히 예약건수도 줄어들고 예약할 때 방문객 수도 현저히 적습니다.

##### 4. 시간대별 예약 방문객 정보

다른 데이터보다 가장 유의미할 것으로 생각이 듭니다. 왜냐하면 예약방문객, 심지어 예약할 떄 방문할 인원에 대한 정보이기 때문에 실제로 방문을 한 사람이 아니기 때문에 상황에 따라 크게 의미가 없는 데이터로 볼 수 있습니다.

따라서 언제 예약을 하는 지에 대한 정보가 더욱 유의미 할 수 있습니다.
예약할 때의 인원수보다 예약 건수 자체로 세는 것이 더욱 의미있을 수 있습니다.

In [ ]:
air_reserve["reserve_datetime_hour"] = air_reserve["reserve_datetime"].dt.hour
air_reserve["visit_datetime_hour"] = air_reserve["visit_datetime"].dt.hour

In [ ]:
visit_hour_visitors = pd.pivot_table(air_reserve, index='visit_datetime_hour', values='reserve_visitors', aggfunc='sum').reset_index()
reserve_hour_visitors = pd.pivot_table(air_reserve, index='reserve_datetime_hour', values='reserve_visitors', aggfunc='sum').reset_index()

print(f"visit_hour_visitors has {len(visit_hour_visitors)} rows.")
print(f"reserve_hour_visitors has {len(reserve_hour_visitors)} rows.")


display(visit_hour_visitors)
display(reserve_hour_visitors)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_hour_visitors, x="visit_datetime_hour", y="reserve_visitors", ax=axs[0], palette=colors, zorder=1)
axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit Date time(hr) / Visitors")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height+1500), ha = 'center', color = 'k', style='italic')

axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

sns.lineplot(data=visit_hour_visitors, x="visit_datetime_hour", y="reserve_visitors", ax=axs[0], color='darkgrey', zorder=2)



sns.barplot(data=reserve_hour_visitors, x="reserve_datetime_hour", y="reserve_visitors", ax=axs[1], palette=colors, zorder=1)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve Date time(hr) / Visitors")

sns.lineplot(data=reserve_hour_visitors, x="reserve_datetime_hour", y="reserve_visitors", ax=axs[1], color='darkgrey', zorder=2)

for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height+1500), ha = 'center', color = 'k', style='italic')


axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

보통 예약할 때의 시간(`reserve_datetime`)은 17, 18시에 몰려있고 예약할 때의 방문시간(`visit_datetime`)은 18, 19시에 몰려있는 경향이 있습니다.

지금까지 데이터를 살펴보니 방문예정객수(`reserve_visitors`)를 살펴보기 보다는 예약 건수로 살펴보는 것이 더욱 의미가 있을 것 같습니다.

요일별, 월별, 계절, 시간대별로 예약 수 분석을 해보겠습니다.

#### 예약수 정보


1. 요일별 예약객 정보: 주말이나 공휴일
2. 월별 예약객 정보: 계절이랑 비슷할 것
3. 계절별 예약객 정보: 제철음식, 음식장르와 관련이 있을 것 같음
4. 시간대별: 저녁시간, 점심시간, ...

In [ ]:
air_reserve["reserve_count"] = 1
total_reserve = pd.pivot_table(air_reserve, index='visit_datetime', values='reserve_count', aggfunc='sum').reset_index()
total_reserve.head(10)

##### 1. 요일별 예약 수 정보

In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
visit_day_reserve_count = pd.pivot_table(air_reserve, index='visit_datetime_day', values='reserve_count', aggfunc='sum').reindex(days).reset_index()
reserve_day_reserve_count = pd.pivot_table(air_reserve, index='reserve_datetime_day', values='reserve_count', aggfunc='sum').reindex(days).reset_index()

print(f"visit_day_reserve_count has {len(visit_day_reserve_count)} rows.")
print(f"reserve_day_reserve_count has {len(reserve_day_reserve_count)} rows.")


display(visit_day_reserve_count)
display(reserve_day_reserve_count)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_day_reserve_count, x="visit_datetime_day", y="reserve_count", ax=axs[0], palette=colors)

axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit day / Reservations") # 해당요일에 방문하겠다고 한 예약수 
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height+1000), ha = 'center', color = 'k', style='italic')

axs[0].patches[4].set_linewidth('2')
axs[0].patches[4].set_edgecolor('darkorange')
axs[0].patches[5].set_linewidth('2')
axs[0].patches[5].set_edgecolor('darkorange')

axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

sns.barplot(data=reserve_day_reserve_count, x="reserve_datetime_day", y="reserve_count", ax=axs[1], palette=colors)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve day / Reservations") # 예약이 온 요일
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height+1000), ha = 'center', color = 'k', style='italic')

axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

##### 월별 예약 수 정보



In [ ]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
visit_month_reserve_count = pd.pivot_table(air_reserve, index='visit_datetime_month', values='reserve_count', aggfunc='sum').reindex(months).reset_index()
reserve_month_reserve_count = pd.pivot_table(air_reserve, index='reserve_datetime_month', values='reserve_count', aggfunc='sum').reindex(months).reset_index()

print(f"visit_month_reserve_count has {len(visit_month_reserve_count)} rows.")
print(f"reserve_month_reserve_count has {len(reserve_month_reserve_count)} rows.")


display(visit_month_reserve_count)
display(reserve_month_reserve_count)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_month_reserve_count, x="visit_datetime_month", y="reserve_count", ax=axs[0], palette=colors, zorder=2)
axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit Month / Reservations")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')

axs[0].axhline(y = 2800, color = 'dodgerblue', ls = '--', zorder = 1)
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)


sns.barplot(data=reserve_month_reserve_count, x="reserve_datetime_month", y="reserve_count", ax=axs[1], palette=colors, zorder=2)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve Month / Reservations")
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')

axs[1].axhline(y = 3500, color = 'dodgerblue', ls = '--', zorder = 1)
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

##### 3. 계절별 예약 수 정보

In [ ]:
seasons = ['Spring', 'Summer', 'Fall', 'Winter']
visit_season_visitors_reserve_count = pd.pivot_table(air_reserve, index='visit_datetime_season', values='reserve_count', aggfunc='sum').reindex(seasons).reset_index()
reserve_season_visitors_reserve_count = pd.pivot_table(air_reserve, index='reserve_datetime_season', values='reserve_count', aggfunc='sum').reindex(seasons).reset_index()

print(f"visit_season_visitors_reserve_count has {len(visit_season_visitors_reserve_count)} rows.")
print(f"reserve_season_visitors_reserve_count has {len(reserve_season_visitors_reserve_count)} rows.")


display(visit_season_visitors_reserve_count)
display(reserve_season_visitors_reserve_count)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_season_visitors_reserve_count, x="visit_datetime_season", y="reserve_count", ax=axs[0], palette=colors, zorder=2)
axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit Month / Reservations")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')

axs[0].axhline(y = 2100, color = 'dodgerblue', ls = '--', zorder = 1)
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)


sns.barplot(data=reserve_season_visitors_reserve_count, x="reserve_datetime_season", y="reserve_count", ax=axs[1], palette=colors, zorder=2)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve Month / Reservavtions")
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')

axs[1].axhline(y = 2300, color = 'dodgerblue', ls = '--', zorder = 1)
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

##### 4. 시간대별 예약 수 정보

In [ ]:
visit_hour_reserve_count = pd.pivot_table(air_reserve, index='visit_datetime_hour', values='reserve_count', aggfunc='sum').reset_index()
reserve_hour_reserve_count = pd.pivot_table(air_reserve, index='reserve_datetime_hour', values='reserve_count', aggfunc='sum').reset_index()

print(f"visit_hour_visitors has {len(visit_hour_visitors)} rows.")
print(f"reserve_hour_visitors has {len(reserve_hour_visitors)} rows.")


display(visit_hour_reserve_count)
display(reserve_hour_reserve_count)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_hour_reserve_count, x="visit_datetime_hour", y="reserve_count", ax=axs[0], palette=colors, zorder=1)
axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit Date time(hr) / Reservations")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height), ha = 'center', color = 'k', style='italic')

axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

sns.lineplot(data=visit_hour_reserve_count, x="visit_datetime_hour", y="reserve_count", ax=axs[0], color='slategray', zorder=2)



sns.barplot(data=reserve_hour_reserve_count, x="reserve_datetime_hour", y="reserve_count", ax=axs[1], palette=colors, zorder=1)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve Date time(hr) / Reservations")

sns.lineplot(data=reserve_hour_reserve_count, x="reserve_datetime_hour", y="reserve_count", ax=axs[1], color='slategray', zorder=2)

for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height), ha = 'center', color = 'k', style='italic')


axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

### 2-2 Analysis of `hpg_reserve`


In [ ]:
hpg_reserve.head()

In [ ]:
hpg_reserve.describe()

In [ ]:
hpg_reserve.isnull().sum()

In [ ]:
hpg_reserve.dtypes

In [ ]:
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])

In [ ]:
hpg_reserve["reserve_count"] = 1
total_reserve = pd.pivot_table(hpg_reserve, index='visit_datetime', values='reserve_count', aggfunc='sum').reset_index()
total_reserve.head(10)

#### 예약 수 정보


위를 통해 결과적으로 예약 수 정보를 보는 것이 더 중요하다고 판단했으므로 예약 수 정보에 대한 것만 다뤄보겠습니다.

##### 요일별 예약 수 정보

In [ ]:
visit_datetime_day = hpg_reserve['visit_datetime'].dt.day_name()
reserve_datetime_day = hpg_reserve['reserve_datetime'].dt.day_name()

In [ ]:
hpg_reserve['visit_datetime_day'] = visit_datetime_day
hpg_reserve['reserve_datetime_day'] = reserve_datetime_day

In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
visit_day_reserve_count = pd.pivot_table(hpg_reserve, index='visit_datetime_day', values='reserve_count', aggfunc='sum').reindex(days).reset_index()
reserve_day_reserve_count = pd.pivot_table(hpg_reserve, index='reserve_datetime_day', values='reserve_count', aggfunc='sum').reindex(days).reset_index()

print(f"visit_day_reserve_count has {len(visit_day_reserve_count)} rows.")
print(f"reserve_day_reserve_count has {len(reserve_day_reserve_count)} rows.")


display(visit_day_reserve_count)
display(reserve_day_reserve_count)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_day_reserve_count, x="visit_datetime_day", y="reserve_count", ax=axs[0], palette=colors)

axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit day / Reservations") # 해당요일에 방문하겠다고 한 예약수 
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height), ha = 'center', color = 'k', style='italic')

axs[0].patches[4].set_linewidth('2')
axs[0].patches[4].set_edgecolor('darkorange')
axs[0].patches[5].set_linewidth('2')
axs[0].patches[5].set_edgecolor('darkorange')

axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

sns.barplot(data=reserve_day_reserve_count, x="reserve_datetime_day", y="reserve_count", ax=axs[1], palette=colors)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve day / Reservations") # 예약이 온 요일
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height), ha = 'center', color = 'k', style='italic')

axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

##### 월별 예약 수 정보



In [ ]:
visit_datetime_month = hpg_reserve['visit_datetime'].dt.month_name()
reserve_datetime_month = hpg_reserve['reserve_datetime'].dt.month_name()
hpg_reserve['visit_datetime_month'] = visit_datetime_month
hpg_reserve['reserve_datetime_month'] = reserve_datetime_month

In [ ]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
visit_month_reserve_count = pd.pivot_table(hpg_reserve, index='visit_datetime_month', values='reserve_count', aggfunc='sum').reindex(months).reset_index()
reserve_month_reserve_count = pd.pivot_table(hpg_reserve, index='reserve_datetime_month', values='reserve_count', aggfunc='sum').reindex(months).reset_index()

print(f"visit_month_reserve_count has {len(visit_month_reserve_count)} rows.")
print(f"reserve_month_reserve_count has {len(reserve_month_reserve_count)} rows.")


display(visit_month_reserve_count)
display(reserve_month_reserve_count)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_month_reserve_count, x="visit_datetime_month", y="reserve_count", ax=axs[0], palette=colors, zorder=2)
axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit Month / Reservations")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)


sns.barplot(data=reserve_month_reserve_count, x="reserve_datetime_month", y="reserve_count", ax=axs[1], palette=colors, zorder=2)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve Month / Reservations")
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')

axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

##### 3. 계절별 예약 수 정보

In [ ]:
hpg_reserve['visit_datetime_season'] = ""
hpg_reserve['reserve_datetime_season'] = ""

In [ ]:
hpg_reserve['visit_datetime_season'] = np.where(hpg_reserve['visit_datetime_month'].isin(["March", "April", "May"]), 'Spring', hpg_reserve['visit_datetime_season'])
hpg_reserve['visit_datetime_season'] = np.where(hpg_reserve['visit_datetime_month'].isin(["June", "July", "August"]), 'Summer', hpg_reserve['visit_datetime_season'])
hpg_reserve['visit_datetime_season'] = np.where(hpg_reserve['visit_datetime_month'].isin(["September", "October", "November"]), 'Fall', hpg_reserve['visit_datetime_season'])
hpg_reserve['visit_datetime_season'] = np.where(hpg_reserve['visit_datetime_month'].isin(["December", "January", "February"]), 'Winter', hpg_reserve['visit_datetime_season'])

In [ ]:
hpg_reserve['reserve_datetime_season'] = np.where(hpg_reserve['reserve_datetime_month'].isin(["March", "April", "May"]), 'Spring', hpg_reserve['reserve_datetime_season'])
hpg_reserve['reserve_datetime_season'] = np.where(hpg_reserve['reserve_datetime_month'].isin(["June", "July", "August"]), 'Summer', hpg_reserve['reserve_datetime_season'])
hpg_reserve['reserve_datetime_season'] = np.where(hpg_reserve['reserve_datetime_month'].isin(["September", "October", "November"]), 'Fall', hpg_reserve['reserve_datetime_season'])
hpg_reserve['reserve_datetime_season'] = np.where(hpg_reserve['reserve_datetime_month'].isin(["December", "January", "February"]), 'Winter', hpg_reserve['reserve_datetime_season'])

In [ ]:
seasons = ['Spring', 'Summer', 'Fall', 'Winter']
visit_season_visitors_reserve_count = pd.pivot_table(hpg_reserve, index='visit_datetime_season', values='reserve_count', aggfunc='sum').reindex(seasons).reset_index()
reserve_season_visitors_reserve_count = pd.pivot_table(hpg_reserve, index='reserve_datetime_season', values='reserve_count', aggfunc='sum').reindex(seasons).reset_index()

print(f"visit_season_visitors_reserve_count has {len(visit_season_visitors_reserve_count)} rows.")
print(f"reserve_season_visitors_reserve_count has {len(reserve_season_visitors_reserve_count)} rows.")


display(visit_season_visitors_reserve_count)
display(reserve_season_visitors_reserve_count)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_season_visitors_reserve_count, x="visit_datetime_season", y="reserve_count", ax=axs[0], palette=colors, zorder=2)
axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit Month / Reservations")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')


axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)


sns.barplot(data=reserve_season_visitors_reserve_count, x="reserve_datetime_season", y="reserve_count", ax=axs[1], palette=colors, zorder=2)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve Month / Reservavtions")
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height+500), ha = 'center', color = 'k', style='italic')

axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

##### 4. 시간대별 예약 수 정보

In [ ]:
hpg_reserve["reserve_datetime_hour"] = hpg_reserve["reserve_datetime"].dt.hour
hpg_reserve["visit_datetime_hour"] = hpg_reserve["visit_datetime"].dt.hour

In [ ]:
visit_hour_reserve_count = pd.pivot_table(hpg_reserve, index='visit_datetime_hour', values='reserve_count', aggfunc='sum').reset_index()
reserve_hour_reserve_count = pd.pivot_table(hpg_reserve, index='reserve_datetime_hour', values='reserve_count', aggfunc='sum').reset_index()

print(f"visit_hour_visitors has {len(visit_hour_visitors)} rows.")
print(f"reserve_hour_visitors has {len(reserve_hour_visitors)} rows.")


display(visit_hour_reserve_count)
display(reserve_hour_reserve_count)

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
axs = axes.ravel()
sns.barplot(data=visit_hour_reserve_count, x="visit_datetime_hour", y="reserve_count", ax=axs[0], palette=colors, zorder=1)
axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xticks([])
axs[0].set_xlabel("")
axs[0].set_title("Visit Date time(hr) / Reservations")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height), ha = 'center', color = 'k', style='italic')

axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

sns.lineplot(data=visit_hour_reserve_count, x="visit_datetime_hour", y="reserve_count", ax=axs[0], color='slategray', zorder=2)



sns.barplot(data=reserve_hour_reserve_count, x="reserve_datetime_hour", y="reserve_count", ax=axs[1], palette=colors, zorder=1)
axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=30)
axs[1].set_xlabel("")
axs[1].set_title("Reserve Date time(hr) / Reservations")

sns.lineplot(data=reserve_hour_reserve_count, x="reserve_datetime_hour", y="reserve_count", ax=axs[1], color='slategray', zorder=2)

for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height), ha = 'center', color = 'k', style='italic')


axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

##### * `air_reserve` vs `air_visit_data`

In [ ]:
reserve_id = air_reserve["air_store_id"].to_numpy()
visit_id = air_visit_data["air_store_id"].to_numpy()
print(f"air_reserve_data의 id 갯수: {len(np.unique(air_reserve['air_store_id'].to_numpy()))}")
print(f"air_visit_data의 id 갯수: {len(np.unique(air_visit_data['air_store_id'].to_numpy()))}")

In [ ]:
print(f"{len(set(visit_id) - set(reserve_id))}개의 매장은 정보가 없습니다.") ## 515개의 정보가 없다고 보는 것이 좋을 것 같다.
print(f"len(set(reserve_id) - set(visit_id)) = {len(set(reserve_id) - set(visit_id))}인 것을 보면 air_visit_data는 air_reserve의 모든 매장을 포함합니다.")
print(f"16. 01. 03의 총 방문객: {air_visit_data[air_visit_data['visit_date'] == '2016-01-13']['visitors'].sum()}") ## visit_data의 합
print(f"16. 01. 03의 예약 방문객: {air_reserve.iloc[599:681, 3].sum()}") ## reserve의 합

515개의 예약정보가 없는 매장이 있을 수 있으므로 `air_reserve`에 정보가 있는 매장에 대한 visit_data를 따로 뽑아내겠습니다.

In [ ]:
air_reserved_visit_data = air_visit_data[air_visit_data["air_store_id"].isin(reserve_id)]

In [ ]:
air_reserved_visit_data[air_reserved_visit_data['visit_date'] == '2016-01-13']['visitors'].sum()

두 데이터는 비슷한 분포를 보이지만 그 갯수에서 확연한 차이를 보입니다. 평균적으로 `hpg`에 등록되어있는 데이터들이 훨씬 많은 방문객 수, 예약 수를 보이고 있습니다. 따라서 추후에 데이터를 합치더라도 어떤 서비스에서 온 데이터인지 명시하는 것이 중요할 것으로 예상됩니다.

또한 EDA 중에 노쇼 고객이 있을 것을 염두에 두고 미리 `air_reserve`와 `air_visit_data`를 이용해 구하려고 했지만 노쇼한 인원을 파악하는 것은 불가능해 보입니다. 이유는 `air_reserve`에서 센 방문예정객 수가 `air_visit_data`에서 센 실제 방문객 수보다 많다면 노쇼라고 할 수 있지만 적다고 해서 노쇼를 하지 않았다고는 할 수 없기 떄문입니다. 하지만 이러한 $\frac{\text{실제 방문객}}{\text{방문 예정객}}$의 비율을 이용한다면 유동인구 파악, 또는 위도/경도의 정보와 함께 번화가와 그렇지 시외지역간의 비율차이를 이용해 유의미한 분석을 할 수 있을 것 같습니다.

무엇보다 이런 분석은 예약 데이터만을 모아놓은 데이터셋을 바라보고 가정적으로 생각을 하고 있는 부분이기 때문에 `store_info`에 대한 분석을 필요로 합니다. 따라서 이어지는 과정에서 `store_info`에 관한 EDA를 이어가겠습니다.

### 2-3 Analysis of `air_store_info`


In [ ]:
air_store_info.head()

In [ ]:
air_store_info.describe()

In [ ]:
air_store_info.isnull().sum()

기본적으로 살펴보니 결측치나 기타 이상은 없는 것 같습니다. id는 단지 고유적인 값일 뿐이니 장르(음식 종류)이름부터 하나씩 살펴보겠습니다.

#### 음식 종류(장르) 정보

In [ ]:
air_genre_count = air_store_info["air_genre_name"].value_counts(ascending=True)

In [ ]:
colors = sns.color_palette("pastel", len(air_genre_count))
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(air_genre_count.values, air_genre_count.index, ax=ax, palette=colors)
ax.set_title("Genre")


for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(int(width), (width+5, bottom+0.5), ha = 'center', color = 'k', style='italic')

장르를 살펴보면 이자카야가 가장 많고 그 뒤를 카페, 다이닝 바가 있으며 아시안, 카라오케/파티, 세계요리(international cuisine)이 2개로 최하위에 위치해있습니다. 장르에 관련된 사항은 위에서 방문 예정자, 예약건수를 다룰 때에 hue 값으로 구분해서 사용하면 아주 요긴하게 사용할 수 있을 것 같습니다.

#### 매장 위도, 경도 정보
위도와 경도에 대한 분석을 해보겠습니다.

In [ ]:
air_store_info.head()

In [ ]:
plt.figure(figsize=(12, 12))
g = sns.jointplot(x=air_store_info.latitude.values, y=air_store_info.longitude.values, marker="+",
                  color='dodgerblue', marginal_kws=dict(color='skyblue'))
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()

jointplot을 이용해 산점도와 분포를 보여주는 히스토그램을 그려보았지만 크게 인사이트를 얻기 힘들어 보입니다. 가장 많은 데이터가 경도 140, 위도 36근방에 가장 많이 위치하고 있으니 어떤 곳인지 한번 확인해보겠습니다.

In [ ]:
from collections import Counter

freq_lat = air_store_info[(35 < air_store_info["latitude"]) & (air_store_info["latitude"] < 37)]
freq_long = air_store_info[(139 < air_store_info["longitude"]) & (air_store_info["longitude"] < 141)]

freq_lat_list = freq_lat['air_area_name'].to_numpy()
freq_long_list = freq_long['air_area_name'].to_numpy()

freq_lat_list = [address.split()[0] for address in freq_lat_list]
freq_long_list = [address.split()[0] for address in freq_long_list]

print(Counter(freq_lat_list))
print(Counter(freq_long_list))

위도 36근방, 경도 140근방의 주소의 첫 단어를 통해 어떤 도, 현에 있는 지 뽑아보니 대부분이 도쿄에 위치해있고 시즈오카현, 미야기현, 니이가타현에 조금씩 분포하고 있습니다. 다시 그림을 보면 대부분의 데이터가 아얘 위도 36, 경도 140에 위치하고 있는데 이는 실제 도쿄의 위도, 경도인 (35.652, 139.839)와 매우 비슷한 위치입니다.

위도, 경도에 해당하는 위치에 산점도로 점을 찍고 어떤 종류의 음식을 파는 지 구분하기 위해 그려본 그림입니다.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sns.scatterplot(data=air_store_info, x='latitude', y='longitude', hue='air_genre_name', ax=ax)
ax.legend(loc='lower right')

자세히 보이지는 않지만 도쿄 부근에는 다양한 음식점이 혼재하고 있다는 점, 그리고 또 다른 식당이 혼재하고 있는 위도 35, 경도 135 부근은 오사카라는 점을 알 수 있습니다. (실제 오사카의 위도: 34.41, 경도: 135.30)

산점도를 통해 어떤 곳에 음식점이 많이 위치하고 있는 지에 대한 정보는 알 수 있지만 전반적인 음식 종류의 분포에 대해서는 알아보기가 힘든 단점이 있습니다.

#### 매장 위치 정보

매장 위치 정보를 우선 음식 종류 정보와 연결시켜 분석해보겠습니다. 번거롭지만 매장에 해당하는 음식 종류수를 세어주기 위해 데이터프레임을 하나씩 만들었습니다.

In [ ]:
air_IF_store = air_store_info[air_store_info["air_genre_name"] == "Italian/French"]
air_BC_store = air_store_info[air_store_info["air_genre_name"] == "Bar/Cocktail"]
air_CS_store = air_store_info[air_store_info["air_genre_name"] == "Cafe/Sweets"]
air_CC_store = air_store_info[air_store_info["air_genre_name"] == "Creative cuisine"]
air_DB_store = air_store_info[air_store_info["air_genre_name"] == "Dining bar"]
air_IC_store = air_store_info[air_store_info["air_genre_name"] == "International cuisine"]
air_IZ_store = air_store_info[air_store_info["air_genre_name"] == "Izakaya"]
air_JA_store = air_store_info[air_store_info["air_genre_name"] == "Japanese food"]
air_KA_store = air_store_info[air_store_info["air_genre_name"] == "Karaoke/Party"]
air_YA_store = air_store_info[air_store_info["air_genre_name"] == "Okonomiyaki/Monja/Teppanyaki"]
air_OT_store = air_store_info[air_store_info["air_genre_name"] == "Other"]
air_WE_store = air_store_info[air_store_info["air_genre_name"] == "Western food"]
air_KO_store = air_store_info[air_store_info["air_genre_name"] == "Yakiniku/Korean food"]
air_AS_store = air_store_info[air_store_info["air_genre_name"] == "Asian"]

In [ ]:
air_IF_count = air_IF_store["air_area_name"].value_counts()
air_BC_count = air_BC_store["air_area_name"].value_counts()
air_CS_count = air_CS_store["air_area_name"].value_counts()
air_CC_count = air_CC_store["air_area_name"].value_counts()
air_DB_count = air_DB_store["air_area_name"].value_counts()
air_IC_count = air_IC_store["air_area_name"].value_counts()
air_IZ_count = air_IZ_store["air_area_name"].value_counts()
air_JA_count = air_JA_store["air_area_name"].value_counts()
air_KA_count = air_KA_store["air_area_name"].value_counts()
air_YA_count = air_YA_store["air_area_name"].value_counts()
air_OT_count = air_OT_store["air_area_name"].value_counts()
air_WE_count = air_WE_store["air_area_name"].value_counts()
air_KO_count = air_KO_store["air_area_name"].value_counts()
air_AS_count = air_AS_store["air_area_name"].value_counts()

In [ ]:
count_list = [air_IF_count, air_BC_count, air_CS_count, air_CC_count, air_DB_count, air_IC_count, air_IZ_count, 
              air_JA_count, air_KA_count, air_YA_count, air_OT_count, air_WE_count, air_KO_count, air_AS_count]
name_list = ['Italian/French', 'Bar/Cocktail', 'Cafe/Sweets', 'Creative cuisine', 'Dining bar', 'International cuisine', 
             'Izakaya', 'Japanese food', 'Karaoke/Party', 'Okonomiyaki/Monja/Teppanyaki', 'Other', 
             'Western food', 'Yakiniku/Korean food', 'Asian']

fig, axes = plt.subplots(figsize=(25, 50), nrows=7, ncols=2)
axs = axes.ravel()
for i, ax in enumerate(axs):
    sns.barplot(y=count_list[i].index, x=count_list[i].values, ax=ax)
    ax.set_title(f"{name_list[i]}")
    for p in ax.patches:
        left, bottom, width, height = p.get_bbox().bounds
        ax.annotate(int(width), (width, bottom+0.5), ha = 'center', color = 'k', style='italic')

fig.tight_layout()

위와 같은 막대 그래프로 봤을 때, 특히나 카페, 다이닝 바, 이자카야 등은 여러 도시에 걸쳐 위치하고 있고 세계요리, 카라오케, 아시안의 경우 한 지역에만 위치하고 있음을 알 수 있습니다.

계속 염두에 둬야 하는 것은 air 플랫폼에 등록된 매장에 한해서 모아진 정보이므로 전체적인 매장의 구성은 hpg와 함께 보는 것이 더 좋아보입니다.

각 지역마다 어떤 종류의 음식점이 몇개씩 있는지에 대한 분포를 알아보기 위해 히트맵을 이용해보겠습니다.

In [ ]:
area_genre = pd.pivot_table(air_store_info, index='air_area_name', columns='air_genre_name', aggfunc='size').fillna(0).astype(np.int8)

In [ ]:
area_genre.head()

In [ ]:
fig, ax = plt.subplots(figsize=(25, 25))
sns.heatmap(area_genre, ax=ax, linewidths=2.5, annot=True, annot_kws={"size": 10, "fontweight": 'bold'}, cmap="Greens", fmt="d")

각 지역별로 어떤 종류의 음식을 파는 매장이 많이 놓여있는 지 볼 수 있습니다. 반대로 음식 종류의 지역적인 분포를 알고 싶다면 데이터프레임의 전치(transpose)를 이용하면 좀 더 쉽게 볼 수 있습니다.

### 2-4 Analysis of hpg_store_info

In [ ]:
hpg_store_info.head()

In [ ]:
hpg_store_info.describe()

In [ ]:
hpg_store_info.isnull().sum()

위와 같은 순서로 장르(음식 종류)이름부터 하나씩 살펴보겠습니다.

#### 음식 종류(장르) 정보

In [ ]:
hpg_genre_count = hpg_store_info["hpg_genre_name"].value_counts(ascending=True)

In [ ]:
hpg_genre_count.index.to_numpy()

살펴보니 air의 경우보다 더 많은 종류의 음식이 담겨있습니다.

In [ ]:
colors = sns.color_palette("pastel", len(air_genre_count))
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(x=hpg_genre_count.values, y=hpg_genre_count.index, ax=ax, palette=colors)
ax.set_title("Genre")


for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(int(width), (width+20, bottom+0.5), ha = 'center', color = 'k', style='italic')

장르를 살펴보면 일본식이 가장 많고 그 뒤를 세계요리(international cuisine)이 creation(창작요리)가 위치해있습니다. 최하위에는 스페인/지중해 요리, 대만/홍콩 요리, 상하이 음식 순으로 놓여있습니다. 

살펴보면 우동/소바, 스시의 경우 등을 별도로 분류해 일식에 넣지 않았다는 점, air와 달리 cafe와 sweets가 빠져있다는 점 등을 주목해서 보면 좋을 것 같습니다.

이 경우에도 hue 값으로 구분해서 사용하면 아주 요긴하게 사용할 수 있을 것 같습니다.

#### 매장 위도, 경도 정보
위도와 경도에 대한 분석을 해보겠습니다.

In [ ]:
plt.figure(figsize=(12, 12))
g = sns.jointplot(x=hpg_store_info.latitude.values, y=hpg_store_info.longitude.values, marker="+",
                  color='dodgerblue', marginal_kws=dict(color='skyblue'))
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()

위와 비슷하게 장 많은 데이터가 경도 140, 위도 36 근방인 도쿄, 그리고 경도 135, 위도 35근방인 오사카에 많이 모여있는 것을 알 수 있습니다.

위도 36근방, 경도 140근방의 주소의 첫 단어를 통해 어떤 도, 현에 있는 지 뽑아보니 대부분이 도쿄에 위치해있고 시즈오카현, 미야기현, 니이가타현에 조금씩 분포하고 있습니다. 다시 그림을 보면 대부분의 데이터가 아얘 위도 36, 경도 140에 위치하고 있는데 이는 실제 도쿄의 위도, 경도인 (35.652, 139.839)와 매우 비슷한 위치입니다.

위도, 경도에 해당하는 위치에 산점도로 점을 찍고 어떤 종류의 음식을 파는 지 구분하기 위해 그려본 그림입니다.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
sns.scatterplot(data=hpg_store_info, x='latitude', y='longitude', hue='hpg_genre_name', ax=ax)
ax.legend(loc='lower right')

이 경우도 air의 경우와 비슷하게 음식점의 전반적인 분포, 음식점이 많이 몰린 곳에는 종류도 다양할 것으로 추측된다는 가정을 생각할 수 있습니다.

#### 매장 위치 정보

매장 위치 정보를 우선 음식 종류 정보와 연결시켜 분석해보겠습니다. 번거롭지만 매장에 해당하는 음식 종류수를 세어주기 위해 데이터프레임을 하나씩 만들었습니다.

In [ ]:
hpg_JS_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Japanese style"]
hpg_IT_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Italian"]
hpg_IC_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "International cuisine"]
hpg_GM_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Grilled meat"]
hpg_CR_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Creation"]
hpg_SH_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Shabu-shabu/Sukiyaki"]
hpg_KC_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Korean cuisine"]
hpg_CJ_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Creative Japanese food"]
hpg_KA_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Japanese cuisine/Kaiseki"]
hpg_SE_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Seafood"]
hpg_JG_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Japanese food in general"]
hpg_PA_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Party"]
hpg_YA_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Okonomiyaki/Monja/Teppanyaki"]
hpg_SU_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Sushi"]
hpg_SI_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Spain Bar/Italian Bar"]
hpg_CG_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Chinese general"]
hpg_BI_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Bistro"]
hpg_FR_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "French"]
hpg_KA_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Karaoke"]
hpg_PP_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Pasta/Pizza"]
hpg_AS_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Cafe"]
hpg_CA_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Sweets"]
hpg_SHG_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Steak/Hamburger/Curry"]
hpg_TV_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Thai/Vietnamese food"]
hpg_WE_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Western food"]
hpg_TH_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Taiwanese/Hong Kong cuisine"]
hpg_CF_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Cantonese food"]
hpg_BC_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Bar/Cocktail"]
hpg_DD_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Dim Sum/Dumplings"]
hpg_AB_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Amusement bar"]
hpg_SF_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Sichuan food"]
hpg_SHF_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Shanghai food"]
hpg_SM_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Spain/Mediterranean cuisine"]
hpg_US_store = hpg_store_info[hpg_store_info["hpg_genre_name"] == "Udon/Soba"]

In [ ]:
hpg_JS_count = hpg_JS_store["hpg_area_name"].value_counts()
hpg_IT_count = hpg_IT_store["hpg_area_name"].value_counts()
hpg_IC_count = hpg_IC_store["hpg_area_name"].value_counts()
hpg_GM_count = hpg_GM_store["hpg_area_name"].value_counts()
hpg_CR_count = hpg_CR_store["hpg_area_name"].value_counts()
hpg_SH_count = hpg_SH_store["hpg_area_name"].value_counts()
hpg_KC_count = hpg_KC_store["hpg_area_name"].value_counts()
hpg_CJ_count = hpg_CJ_store["hpg_area_name"].value_counts()
hpg_KA_count = hpg_KA_store["hpg_area_name"].value_counts()
hpg_SE_count = hpg_SE_store["hpg_area_name"].value_counts()
hpg_JG_count = hpg_JG_store["hpg_area_name"].value_counts()
hpg_PA_count = hpg_PA_store["hpg_area_name"].value_counts()
hpg_YA_count = hpg_YA_store["hpg_area_name"].value_counts()
hpg_SU_count = hpg_SU_store["hpg_area_name"].value_counts()
hpg_SI_count = hpg_SI_store["hpg_area_name"].value_counts()
hpg_CG_count = hpg_CG_store["hpg_area_name"].value_counts()
hpg_BI_count = hpg_BI_store["hpg_area_name"].value_counts()
hpg_FR_count = hpg_FR_store["hpg_area_name"].value_counts()
hpg_KA_count = hpg_KA_store["hpg_area_name"].value_counts()
hpg_PP_count = hpg_PP_store["hpg_area_name"].value_counts()
hpg_AS_count = hpg_AS_store["hpg_area_name"].value_counts()
hpg_CA_count = hpg_CA_store["hpg_area_name"].value_counts()
hpg_SHG_count = hpg_SHG_store["hpg_area_name"].value_counts()
hpg_TV_count = hpg_TV_store["hpg_area_name"].value_counts()
hpg_WE_count = hpg_WE_store["hpg_area_name"].value_counts()
hpg_TH_count = hpg_TH_store["hpg_area_name"].value_counts()
hpg_CF_count = hpg_CF_store["hpg_area_name"].value_counts()
hpg_BC_count = hpg_BC_store["hpg_area_name"].value_counts()
hpg_DD_count = hpg_DD_store["hpg_area_name"].value_counts()
hpg_AB_count = hpg_AB_store["hpg_area_name"].value_counts()
hpg_SF_count = hpg_SF_store["hpg_area_name"].value_counts()
hpg_SHF_count = hpg_SHF_store["hpg_area_name"].value_counts()
hpg_SM_count = hpg_SM_store["hpg_area_name"].value_counts()
hpg_US_count = hpg_US_store["hpg_area_name"].value_counts()

In [ ]:
hpg_JS_count.index

In [ ]:
count_list = [hpg_JS_count, hpg_IT_count, hpg_IC_count, hpg_GM_count, hpg_CR_count, hpg_SH_count, hpg_KC_count, 
              hpg_CJ_count, hpg_KA_count, hpg_SE_count, hpg_JG_count, hpg_PA_count, hpg_YA_count, hpg_SU_count,
              hpg_SI_count, hpg_CG_count, hpg_BI_count, hpg_FR_count, hpg_KA_count, hpg_PP_count, hpg_AS_count,
              hpg_CA_count, hpg_SHG_count, hpg_TV_count, hpg_WE_count, hpg_TH_count, hpg_CF_count, hpg_BC_count,
              hpg_DD_count, hpg_AB_count, hpg_SF_count, hpg_SHF_count, hpg_SM_count, hpg_US_count]

name_list = hpg_store_info["hpg_genre_name"].unique()

fig, axes = plt.subplots(figsize=(50, 40), nrows=6, ncols=6)
axs = axes.ravel()
for i, ax in enumerate(axs):
    if i >= len(count_list): continue
    sns.barplot(y=count_list[i].index, x=count_list[i].values, ax=ax)
    ax.set_title(f"{name_list[i]}")
    for p in ax.patches:
        left, bottom, width, height = p.get_bbox().bounds
        ax.annotate(int(width), (width, bottom+0.5), ha = 'center', color = 'k', style='italic')

fig.tight_layout()

데이터도 많고 종류도 너무 많다보니 실제적으로 상당히 보기가 어렵습니다. 전체적인 갯수와 분포를 살펴보는 정도로 참고하는 게 좋을 것 같습니다. 몇개의 데이터는 한 매장만 있는 경우도 많으니 적당히 카테고리를 합쳐주는 방향도 생각해봐야 할 것 같습니다.

In [ ]:
hpg_genre = pd.pivot_table(hpg_store_info, index='hpg_area_name', columns='hpg_genre_name', aggfunc='size').fillna(0).astype(np.int8)

In [ ]:
hpg_genre.head()

In [ ]:
fig, ax = plt.subplots(figsize=(25, 25))
sns.heatmap(hpg_genre, ax=ax, linewidths=2.5, annot=True, annot_kws={"size": 10, "fontweight": 'bold'}, cmap="Greens", fmt="d")

hpg도 heatmap을 통해 각 지역별로 어떤 종류의 음식을 파는 매장이 많이 놓여있는 지 볼 수 있습니다. 반대로 음식 종류의 지역적인 분포를 알고 싶다면 데이터프레임의 전치(transpose)를 이용하면 좀 더 쉽게 볼 수 있습니다.

In [ ]:
len(sample_submission)

### 2-5 Analysis of `air_visit_data`

air 플랫폼에 등록된 가게에 실제로 방문한 방문자수에 대해 EDA를 해보겠습니다.

In [ ]:
air_visit_data.head()

In [ ]:
air_visit_data.describe()

In [ ]:
air_visit_data.isnull().sum()

우선 위에서 했던 것처럼 `visit_date`를 datetime 형식으로 바꿔주겠습니다.

In [ ]:
air_visit_data['visit_date'] = pd.to_datetime(air_visit_data['visit_date'])

In [ ]:
air_visit_data_day = air_visit_data['visit_date'].dt.day_name()
air_visit_data_month = air_visit_data['visit_date'].dt.month_name()

air_visit_data["visit_date_day"] = air_visit_data_day
air_visit_data["visit_date_month"] = air_visit_data_month

air_visit_data.head()

In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

day_visitor_count = pd.pivot_table(air_visit_data, index='visit_date_day', values='visitors', aggfunc='sum').reindex(days).reset_index()
month_visitor_count = pd.pivot_table(air_visit_data, index='visit_date_month', values='visitors', aggfunc='sum').reindex(months).reset_index()

In [ ]:
display(day_visitor_count)
display(month_visitor_count)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 15))
axs = axes.ravel()
sns.barplot(data=day_visitor_count, x="visit_date_day", y="visitors", ax=axs[0], zorder=2)
sns.lineplot(data=day_visitor_count, x="visit_date_day", y="visitors", ax=axs[0], zorder=1, 
             linewidth=2, linestyle='--', color='lightcoral')
axs[0].set_title("Day / Visitor")
axs[0].set_ylabel("Visitor")
axs[0].set_xlabel("Day")
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height), ha = 'center', color = 'k', style='italic')

axs[0].patches[4].set_linewidth('4')
axs[0].patches[4].set_edgecolor('darkorange')
axs[0].patches[5].set_linewidth('4')
axs[0].patches[5].set_edgecolor('darkorange')

sns.barplot(data=month_visitor_count, x="visit_date_month", y="visitors", ax=axs[1], zorder=2)
sns.lineplot(data=month_visitor_count, x="visit_date_month", y="visitors", ax=axs[1], zorder=1,
             linewidth=2, linestyle='--', color='lightcoral')
axs[1].set_title("Month / Visitor")
axs[1].set_ylabel("Visitor")
axs[1].set_xlabel("Month")
x_tick_labels = [t.get_text() for t in axs[1].get_xticklabels()]
axs[1].set_xticklabels(x_tick_labels, rotation=45)
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height), ha = 'center', color = 'k', style='italic')  

axs[1].patches[4].set_linewidth('4')
axs[1].patches[4].set_edgecolor('tomato')
axs[1].patches[5].set_linewidth('4')
axs[1].patches[5].set_edgecolor('tomato')  

위에서 살펴본 것과 비슷하게 실제 방문자 또한 금요일, 토요일에 많이 있음을 알 수 있습니다. 또한 데이터 구성에 있어서 2017년 4월을 기준으로 나뉘기 때문에 5, 6월에 해당하는 데이터가 적은 것으로 생각이 듭니다. (추가 파악 필요)

### 2-6 Analysis of the rest
`store_id_relation`, `date_info`, `sample_submission` 에 대한 간단한 분석을 해보겠습니다. `store_id_relation`, `date_info` 데이터는 실제 아주 중요한 특성값을 보유하기 보다는 데이터 분석에 도움이 되는 정보들을 담고 있습니다. 따라서 어떤 모습을 보이고 있는지 어떻게 활용할 수 있는지에 초점을 두고 살펴보겠습니다.

`sample_submission`을 통해서는 개략적인 제출해야 되는 형식을 보면서 어떤식으로 데이터를 구성해야하는지 알아보겠습니다.

In [ ]:
store_id_relation.head()

store_id_relation은 air와 hpg 둘다 사용하는 매장에 한해서 어떤 id를 각각 가지는 지 보여주고 있습니다. 테스트 셋에서는 air에 대한 데이터만 가지고 있으므로 이에 맞게 둘다 포함되는 hpg만 air_id로 바꿔 데이터 셋을 구성하는 방향으로 생각하면 될 것 같습니다.

In [ ]:
date_info.head()

date_info는 해당하는 날짜가 무슨 요일이고 공휴일인지 표기를 해주고 있습니다. 요일에 관해서는 pandas의 메소드로 처리를 했으므로 공휴일 유무를 데이터에 추가하는 방향으로 활용할 수 있을 것 같습니다.

In [ ]:
sample_submission.head()

캐글 대회에 파일을 제출할 때 어떤식으로 구성되어야 하는지에 대해 보여주고 있습니다.

In [ ]:
test = sample_submission.copy()
test.head()

[Reference](https://github.com/junaidnasirkhan/Restaurant-Visitor-Forecasting)

데이터를 준비하는 과정에서 merge의 on등을 이용해 겹치는 부분에 한해 쉽게 붙일 수 있는 점이 인상적이여서 많은 참고를 하게 되었습니다.

테스트 셋이 air로만 이루어져있기 때문에, air에 관한 정보를 찾아서 빼내주어야 한다는 인사이트를 늦게 얻어 많이 헤맸습니다. 그 과정에서 많은 도움이 됐습니다. 

가장 많이 참고한 부분은 데이터를 준비하는 과정에서 air_data를 만드는 부분이였으며 기타 total_reserve_data 또는 이를 이용해 다양한 피쳐를 추가해주는 부분은 최대한 해커톤 조 내에서 하려고 노력했습니다.


## 학습용 데이터 준비

*(별도의 노트북에서 진행해 아래에 모듈 및 데이터를 다운로드 받는 부분이 다시 구현되있습니다.)*

### Kaggle에서 데이터 다운로드 받기

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
ls -1ha kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

In [ ]:
!kaggle competitions download -c recruit-restaurant-visitor-forecasting

In [ ]:
air_reserve = pd.read_csv("air_reserve.csv.zip")
hpg_reserve = pd.read_csv("hpg_reserve.csv.zip")
air_store_info = pd.read_csv("air_store_info.csv.zip")
hpg_store_info = pd.read_csv("hpg_store_info.csv.zip")
air_visit_data = pd.read_csv("air_visit_data.csv.zip")  
store_id_relation = pd.read_csv("store_id_relation.csv.zip")
date_info = pd.read_csv("date_info.csv.zip")
sample_submission = pd.read_csv("sample_submission.csv.zip")

### air_data

`air_data`를 만드는 과정입니다.
1. `air_visit_data`에 `air_store_info`를 합칩니다. 이 때 `air_store_id`를 기준으로 하며, `air_visit_data`에 있는 고유값들을 기준으로 합쳐줍니다.
2. `date_info`에 해당하는 값을 합쳐줍니다.
3. `visit_date` 기준으로 정렬을 하고, date에서 뽑아낼 수 있는 날짜에 대한 정보를 활용해 `air_data`를 가공해줍니다.

In [ ]:
air_data = pd.merge(air_visit_data, air_store_info, how='left', on=['air_store_id']) # merging dataframes
date_info.rename(columns={'calendar_date':'visit_date'},inplace=True)  # renaming columns
air_data = pd.merge(air_data,date_info,how='left', on=['visit_date'])
air_data.sort_values(by='visit_date',ignore_index=True,inplace=True)
air_data['visit_date'] = pd.to_datetime(air_data['visit_date'])
air_data['day'] = air_data['visit_date'].dt.day
air_data['dow'] = air_data['visit_date'].dt.isocalendar().week ## weekday -> weekofyear
air_data['year'] = air_data['visit_date'].dt.year
air_data['month'] = air_data['visit_date'].dt.month
air_data['week'] = air_data['visit_date'].dt.isocalendar().week
air_data['quarter'] = air_data['visit_date'].dt.quarter
air_data['visit_date'] = air_data['visit_date'].dt.date
air_data.to_csv('air_data.csv',index=False)

`air_reserve_data`를 만드는 과정입니다.
1. `air_reserve`에 `air_store_info`를 합칩니다. 이 때 `air_store_id`를 기준으로 하며, `air_visit_data`에 있는 고유값들을 기준으로 합쳐줍니다.

2. date에서 뽑아낼 수 있는 날짜에 대한 정보를 활용해 `air_data`를 가공해줍니다.
3. 추가적으로 방문예정시각과 예약시각의 차이를 `res_vis_diff` 라는 컬럼에 입력해줍니다.

In [ ]:
# preprocessing AIR reservation data
air_reserve_data = pd.merge(air_reserve, air_store_info, how='left', on=['air_store_id'])
air_reserve_data.head()
air_reserve_data['visit_datetime'] = pd.to_datetime(air_reserve_data['visit_datetime'])
air_reserve_data['visit_hour'] = air_reserve_data['visit_datetime'].dt.hour
air_reserve_data['visit_date'] = air_reserve_data['visit_datetime'].dt.date
air_reserve_data['reserve_datetime'] = pd.to_datetime(air_reserve_data['reserve_datetime'])
air_reserve_data['reserve_hour'] = air_reserve_data['reserve_datetime'].dt.hour
air_reserve_data['reserve_date'] = air_reserve_data['reserve_datetime'].dt.date
#calculate reservation time difference 
air_reserve_data['res_vis_diff'] = air_reserve_data.apply(
        lambda d: (d['visit_date'] - d['reserve_date']).days, axis=1)
air_reserve_data.rename(columns={'reserve_visitors':'air_reserve_visitors'},inplace=True)
air_reserve_data.to_csv('air_reserve_data.csv',index=False)

### hpg_reserve_data

`hpg_reserve_data`를 만드는 과정입니다.
1. `hpg_reserve`에 `store_id_relation`를 합칩니다. 이 때 `hpg_store_id`를 기준으로 합쳐줍니다. 이런 방식을 통해 `store_id_relation`과 동시에 `hpg_reserve`에 존재하는 식당을 대상으로 merge를 하게 됩니다. _(`how=inner` 를 사용)_

`air_data`와 비슷한 과정을 반복합니다.

In [ ]:
hpg_reserve_data = pd.merge(hpg_reserve,store_id_relation,on=['hpg_store_id'],how='inner')
hpg_reserve_data = pd.merge(hpg_reserve_data,hpg_store_info,on=['hpg_store_id'],how='left')
hpg_reserve_data['visit_datetime'] = pd.to_datetime(hpg_reserve_data['visit_datetime'])
hpg_reserve_data['visit_hour'] = hpg_reserve_data['visit_datetime'].dt.hour
hpg_reserve_data['visit_date'] = hpg_reserve_data['visit_datetime'].dt.date
hpg_reserve_data['reserve_datetime'] = pd.to_datetime(hpg_reserve_data['reserve_datetime'])
hpg_reserve_data['reserve_hour'] = hpg_reserve_data['reserve_datetime'].dt.hour
hpg_reserve_data['reserve_date'] = hpg_reserve_data['reserve_datetime'].dt.date
#calculate reserve time difference 
hpg_reserve_data['res_vis_diff'] = hpg_reserve_data.apply(
        lambda r: (r['visit_date'] - r['reserve_date']).days, axis=1)
hpg_reserve_data.rename(columns={'reserve_visitors':'hpg_reserve_visitors'},inplace=True)

hpg_reserve_data.to_csv('hpg_reserve_data.csv',index=False)


In [ ]:
hpg_reserve_data.head()

In [ ]:
hpg_reserve_data = hpg_reserve_data.drop(columns='hpg_store_id', axis=1)
hpg_reserve_data = pd.merge(hpg_reserve_data, air_store_info, on='air_store_id', how='left')
hpg_reserve_data['air_store_id'].replace('(.*)air_(.*)', r'\1both_\2', regex=True, inplace=True) ## 둘다 포함됐음을 명시

hpg_reserve_data.head()

### total_reserve_data

전체 예약정보를 볼 수 있는 total_reserve_data 를 만드는 과정입니다. concat을 통해 합치면서 발생하는 NaN값들은 모두 수치형데이터 또는 범주형 데이터이므로 0으로 처리를 해줬습니다.

또한 위도, 경도 정보가 들어오면서 각각 다른 열로 분류되며 일부가 결측치로 나오는 경우가 있어서 해당하는 값으로 채워줬습니다.

장르네임 같은 경우, hpg에서 결측인 부분이 있어 해당 부분은 air의 장르로 채워줬습니다. 이후 필요없는 열을 버리면서 데이터프레임을 완성합니다.

In [ ]:
total_reserve_data = pd.concat([air_reserve_data, hpg_reserve_data])

total_reserve_data['air_reserve_visitors'].fillna(0, inplace=True)

total_reserve_data['latitude'].fillna(total_reserve_data['latitude_x'], inplace=True)
total_reserve_data['latitude'].fillna(total_reserve_data['latitude_y'], inplace=True)
total_reserve_data['longitude'].fillna(total_reserve_data['longitude_x'], inplace=True)
total_reserve_data['longitude'].fillna(total_reserve_data['longitude_y'], inplace=True)

total_reserve_data['hpg_reserve_visitors'].fillna(0, inplace=True)

total_reserve_data['hpg_genre_name'].fillna(total_reserve_data["air_genre_name"], inplace=True)
total_reserve_data.drop(columns=['hpg_area_name', "latitude_x", "longitude_x", "latitude_y", "longitude_y"], axis=1, inplace=True)

total_reserve_data['both-contained'] = np.where(total_reserve_data['air_store_id'].str.contains('both'), 1, 0) ## 이름을 바꿔주면서 Flag로 사용.
total_reserve_data['air_store_id'].replace('(.*)both_(.*)', r'\1air_\2', regex=True, inplace=True)

total_reserve_data.drop(columns='air_genre_name', axis=1, inplace=True) ## hpg_genre_name에 포함되어있다.
total_reserve_data.drop(columns=['visit_datetime', 'reserve_datetime'], axis=1, inplace=True)

In [ ]:
print('HEAD')
display(total_reserve_data.head())
print('TAIL')
display(total_reserve_data.tail())

air_data에 넣어줄 각 플랫폼 별 전체 예약수를 넣어주고 있습니다.

In [ ]:
total_air_reserve = total_reserve_data.groupby(['air_store_id','visit_date'],as_index=False)['air_reserve_visitors'].sum()
total_hpg_reserve =  total_reserve_data.groupby(['air_store_id','visit_date'],as_index=False)['hpg_reserve_visitors'].sum()

In [ ]:
air_data = pd.merge(air_data,total_air_reserve,on=['air_store_id','visit_date'],how='left')
air_data = pd.merge(air_data,total_hpg_reserve,on=['air_store_id','visit_date'],how='left')

air_data.shape

In [ ]:
air_data.fillna(0, inplace=True)

In [ ]:
air_data.head()

In [ ]:
air_data.to_csv('air_data.csv', index=False)

In [ ]:
from google.colab import files
files.download("air_data.csv")

#### EDA of air_data

air_data에 대한 분석을 간단하게 진행해보겠습니다.

In [ ]:
air_data.head()

In [ ]:
genre_visitors = pd.pivot_table(air_data, index='air_genre_name', values='visitors', aggfunc='sum').reset_index()
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_visitors = pd.pivot_table(air_data, index='day_of_week', values='visitors', aggfunc='sum').reindex(days).reset_index()

In [ ]:
day_visitors

In [ ]:
colors = sns.color_palette("pastel", 7)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 15))
axs = axes.ravel()
sns.barplot(data=genre_visitors, x="air_genre_name", y="visitors", ax=axs[0], palette=colors)

axs[0].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[0].set_xlabel("Genre", fontsize=12, fontweight='light')

axs[0].set_title("Genre / Visitors", fontweight='bold')
for p in axs[0].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[0].annotate(int(height), (left+width/2, height+1000), ha = 'center', color = 'k', style='italic')

x_tick_labels = [t.get_text() for t in axs[0].get_xticklabels()]
axs[0].set_xticklabels(x_tick_labels, rotation=90)

axs[0].patches[7].set_linewidth('2')
axs[0].patches[7].set_edgecolor('darkorange')

axs[0].patches[2].set_linewidth('2')
axs[0].patches[2].set_edgecolor('darkorange')



axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

sns.barplot(data=day_visitors, x="day_of_week", y="visitors", ax=axs[1], palette=colors)

axs[1].set_ylabel("Visitors", fontsize=12, fontweight='light')
axs[1].set_xlabel("Day of Week", fontsize=12, fontweight='light')

axs[1].set_title("Day of week / Visitors", fontweight='bold')
for p in axs[1].patches:
    left, bottom, width, height = p.get_bbox().bounds
    axs[1].annotate(int(height), (left+width/2, height+1000), ha = 'center', color = 'k', style='italic')

axs[1].patches[4].set_linewidth('2')
axs[1].patches[4].set_edgecolor('darkorange')
axs[1].patches[5].set_linewidth('2')
axs[1].patches[5].set_edgecolor('darkorange')

axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

fig.tight_layout()

요일은 예약과 비슷하게 금, 토요일에 가장 방문객이 많습니다. 예약이 많은 만큼 방문객도 많다고 생각할 수 있을 것 같습니다. 음식 종류의 경우 이자카야가 가장 많고 그 뒤로는 카페가 있습니다.

### Training set
위에서 만든 air_data를 train_set으로 사용할 수 있습니다. 따라서 복사를 해주고 필요에 따라 컬럼을 없애서 중복되는 feature가 없게 분류해주고 있습니다.

또한 각 feature별 특성에 따라 범주형 데이터가 되야하는 경우, 라벨링을 하는 전처리를 진행하겠습니다.

In [ ]:
train_set = air_data.copy()

In [ ]:
train_set.head()

In [ ]:
print("Training data duration:{} to {}".format(train_set.visit_date.min(), train_set.visit_date.max()))

In [ ]:
train_set.drop(columns=["visit_date"], axis=1, inplace=True) ## 이미 정보가 저장되어 있으므로

In [ ]:
## 범주형 데이터가 되야하는 부분 ##
train_set['air_store_id'] = train_set['air_store_id'].astype('category').cat.codes ## 이거 주의
train_set['day_of_week'] = train_set['day_of_week'].astype('category').cat.codes
train_set['air_genre_name'] = train_set['air_genre_name'].astype('category').cat.codes
train_set['air_area_name'] = train_set['air_area_name'].astype('category').cat.codes
train_set['day_of_week'] = train_set['day_of_week'].astype('category').cat.codes
train_set['year'] = train_set['year'].astype('category').cat.codes
train_set['month'] = train_set['month'].astype('category').cat.codes
train_set['week'] = train_set['week'].astype('category').cat.codes
train_set['quarter'] = train_set['quarter'].astype('category').cat.codes

In [ ]:
train_set.head()

In [ ]:
train_set.shape

### Submission

`sample_submission` 데이터는 추후에 test_data로 활용할 예정입니다. 따라서 테스트 셋으로 사용 될 것을 염두에 두고 미리 전처리를 해주겠습니다. 기본적으로는 위의 과정을 다시 한번 반복해서 적용시켜 주고 있습니다.

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv.zip")

In [ ]:
sample_submission['air_store_id'] = sample_submission['id'].map(lambda x: '_'.join(x.split('_')[:2]))
sample_submission['visit_date'] = sample_submission['id'].map(lambda x: str(x).split('_')[2])

sample_submission = pd.merge(sample_submission, date_info, how='left', on=['visit_date'])

sample_submission['visit_date'] = pd.to_datetime(sample_submission['visit_date'])
sample_submission['day'] = sample_submission['visit_date'].dt.day
sample_submission['dow'] = sample_submission['visit_date'].dt.weekday
sample_submission['dow_name'] = sample_submission['visit_date'].dt.day_name()
sample_submission['year'] = sample_submission['visit_date'].dt.year
sample_submission['month'] = sample_submission['visit_date'].dt.month
sample_submission['week'] = sample_submission['visit_date'].dt.isocalendar().week
sample_submission['quarter'] = sample_submission['visit_date'].dt.quarter
sample_submission['visit_date'] = sample_submission['visit_date'].dt.date

sample_submission.to_csv('sample_submission.csv',index=False)

sample_submission.shape

In [ ]:
sample_submission.drop(columns='id', axis=1, inplace=True)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission = pd.merge(sample_submission, air_store_info, how='left', on=['air_store_id'])
sample_submission = pd.merge(sample_submission,total_air_reserve,on=['air_store_id','visit_date'],how='left')
sample_submission = pd.merge(sample_submission,total_hpg_reserve,on=['air_store_id','visit_date'],how='left')

In [ ]:
sample_submission.fillna(0, inplace=True)
sample_submission.head()

In [ ]:
sample_submission.drop(columns=['visit_date', 'dow_name'], axis=1, inplace=True)

In [ ]:
print(train_set.shape[1] == sample_submission.shape[1])

### Test set

In [ ]:
test_set = sample_submission.copy()

In [ ]:
test_set.drop(columns='visitors', axis=1, inplace=True)

In [ ]:
print(train_set.shape) ## visitors를 가지고 있으므로 1개 더 많다.
print(test_set.shape)

In [ ]:
## 범주형 데이터가 되야하는 부분 ##
test_set['air_store_id'] = test_set['air_store_id'].astype('category').cat.codes ## 이거 주의
test_set['day_of_week'] = test_set['day_of_week'].astype('category').cat.codes
test_set['air_genre_name'] = test_set['air_genre_name'].astype('category').cat.codes
test_set['air_area_name'] = test_set['air_area_name'].astype('category').cat.codes
test_set['day_of_week'] = test_set['day_of_week'].astype('category').cat.codes
test_set['year'] = test_set['year'].astype('category').cat.codes
test_set['month'] = test_set['month'].astype('category').cat.codes
test_set['week'] = test_set['week'].astype('category').cat.codes
test_set['quarter'] = test_set['quarter'].astype('category').cat.codes

### 학습

다양한 모델과 다양한 방법을 이용해 학습을 진행해보겠습니다.

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train = train_set.drop(columns='visitors')
y_train = train_set.visitors

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(test_set.shape)

#### KNN

거리 개념을 사용하는 KNN의 경우를 고려해 수치형 데이터만을 뽑아내고 스케일링을 진행해주겠습니다.

In [ ]:
X_train_knn = X_train[["latitude", "longitude", "air_reserve_visitors", "hpg_reserve_visitors"]]
test_set_knn = test_set[["latitude", "longitude", "air_reserve_visitors", "hpg_reserve_visitors"]]

std = StandardScaler() 
X_train_knn_std = std.fit_transform(X_train_knn) # 수치형 데이터만
test_set_knn_std = std.transform(test_set_knn)

In [ ]:
print(X_train_knn_std.shape)
print(test_set_knn_std.shape)

Grid Search를 통해 좋은 파라미터를 찾아보겠습니다.

In [ ]:
from sklearn.metrics import mean_squared_log_error, make_scorer
scorer = make_scorer(mean_squared_log_error)

# hyperparameters
parameters = {'n_neighbors':range(1, 20, 2)}

# model
knn = KNeighborsRegressor(n_jobs=-1)

# cross validation
knn_cv = GridSearchCV(estimator=knn,
                      param_grid=parameters,
                      scoring=scorer,
                      n_jobs=-1,
                      cv=3,
                      verbose=10,
                      return_train_score=True)

knn_cv.fit(X_train_knn_std, y_train)

In [ ]:
knn_cv.best_params_

아무래도 파라미터를 잘 찾지 못한 것 같습니다. 파라미터를 8정도로 두고 해보겠습니다.

In [ ]:
knn = KNeighborsRegressor(n_neighbors=8, n_jobs=-1)
knn.fit(X_train_knn_std, y_train)

In [ ]:
y_pred = knn.predict(X_train_knn_std)

In [ ]:
np.sqrt(mean_squared_log_error(y_train, y_pred))

In [ ]:
print(f"true values:{y_train.values[:5]}")
print(f"predicted values: {y_pred[:5]}")

In [ ]:
y_test = knn.predict(test_set_knn_std)

In [ ]:
for_submission = pd.read_csv("sample_submission.csv.zip")

In [ ]:
for_submission['visitors'] = y_test

In [ ]:
for_submission.head()

In [ ]:
for_submission.to_csv('submission_knn_scaled_8_neighbors.csv', index=False)

In [ ]:
files.download("submission_knn_scaled_8_neighbors.csv")

#### SGD Regression

SGD 같은 경우에도 step-size가 길이에 따라 다르게 움직이므로 위와 같이 수치형 데이터를 스케일링을 한 데이터를 사용하겠습니다.

In [ ]:
std = StandardScaler() 
X_train_std = std.fit_transform(X_train_knn) # 수치형 데이터만
test_std = std.transform(test_set_knn)

In [ ]:
sgd = SGDRegressor(alpha=0.001)
sgd.fit(X_train_knn_std, y_train)

In [ ]:
y_pred = sgd.predict(X_train_knn_std)
from sklearn.metrics import mean_squared_log_error
np.sqrt(mean_squared_log_error(y_train, y_pred))

In [ ]:
print(f"true values:{y_train.values[:5]}")
print(f"predicted values: {y_pred[:5]}")

In [ ]:
y_test = sgd.predict(test_set_knn_std)

In [ ]:
for_submission = pd.read_csv("sample_submission.csv.zip")

In [ ]:
for_submission['visitors'] = y_test

In [ ]:
for_submission.head()

In [ ]:
for_submission.to_csv('submission_sgd_ex_sc.csv', index=False)

In [ ]:
files.download("submission_sgd_ex_sc.csv")

#### DT

트리는 스케일링에 따라 변하지 않으므로 그대로의 데이터를 사용하겠습니다.

In [ ]:
dt = DecisionTreeRegressor(max_depth = 30, min_samples_split = 100)
dt.fit(X_train, y_train)

In [ ]:
y_pred = dt.predict(X_train)
from sklearn.metrics import mean_squared_log_error
np.sqrt(mean_squared_log_error(y_train, y_pred))

In [ ]:
print(f"true values:{y_train.values[:5]}")
print(f"predicted values: {y_pred[:5]}")

In [ ]:
y_test = dt.predict(test_set)

In [ ]:
for_submission = pd.read_csv("sample_submission.csv.zip")

In [ ]:
for_submission['visitors'] = y_test

In [ ]:
for_submission.head()

In [ ]:
for_submission.to_csv('submission_dt.csv', index=False)

In [ ]:
files.download("submission_dt.csv")

#### RF

DT를 앙상블하는 랜덤포레스트를 이용해보겠습니다.

In [ ]:
rf = RandomForestRegressor(max_depth = 30, n_estimators = 100, n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_train)
from sklearn.metrics import mean_squared_log_error
np.sqrt(mean_squared_log_error(y_train, y_pred))

In [ ]:
print(f"true values:{y_train.values[:5]}")
print(f"predicted values: {y_pred[:5]}")

In [ ]:
y_test = rf.predict(test_set)

In [ ]:
for_submission = pd.read_csv("sample_submission.csv.zip")

In [ ]:
for_submission['visitors'] = y_test

In [ ]:
for_submission.head()

In [ ]:
for_submission.to_csv('submission_rf.csv', index=False)

In [ ]:
from google.colab import files
files.download("submission_rf.csv")

#### 앙상블

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

gboost = GradientBoostingRegressor(random_state=2019)
xgboost = xgb.XGBRegressor(random_state=2019, n_estimators=80, learning_rate=0.05, max_depth=7)
lightgbm = lgb.LGBMRegressor(random_state=2019,  n_estimators=80, learning_rate=0.05, num_leaves=60)

models = [{'model':gboost, 'name':'GradientBoosting'}, {'model':xgboost, 'name':'XGBoost'},
          {'model':lightgbm, 'name':'LightGBM'}]

In [ ]:
def AveragingBlending(models, x, y, sub_x):
    for m in models : 
        m['model'].fit(x.values, y)
    
    predictions = np.column_stack([
        m['model'].predict(sub_x.values) for m in models
    ])
    return np.mean(predictions, axis=1)

In [ ]:
y_pred = AveragingBlending(models, X_train, y_train, test_set)
print(len(y_pred))
y_pred

In [ ]:
y_pred.shape

In [ ]:
for_submission = pd.read_csv("sample_submission.csv.zip")

In [ ]:
for_submission['visitors'] = y_pred

In [ ]:
for_submission.head()

In [ ]:
for_submission.to_csv('submission_ensemble_gxl.csv', index=False)

In [ ]:
files.download("submission_ensemble_gxl.csv")

##### 더 적은 특성값으로

지나치게 범주가 많아 학습에 방해가 될 수도 있는 매장명과 지역명을 버린 데이터로 위의 앙상블 학습을 진행해보겠습니다. 

In [ ]:
X_train_ex = X_train.drop(columns=["air_store_id", "air_area_name"])
test_set_ex = test_set.drop(columns=["air_store_id", "air_area_name"])

In [ ]:
print(X_train_ex.shape)
print(test_set_ex.shape)

In [ ]:
y_pred = AveragingBlending(models, X_train_ex, y_train, test_set_ex)
print(len(y_pred))
y_pred

In [ ]:
y_pred.shape

In [ ]:
for_submission = pd.read_csv("sample_submission.csv.zip")

In [ ]:
for_submission['visitors'] = y_pred

In [ ]:
for_submission.head()

In [ ]:
for_submission.to_csv('submission_ensemble_gxl_ex.csv', index=False)

In [ ]:
from google.colab import files
files.download("submission_ensemble_gxl_ex.csv")

In [ ]:
X_train_ex_2 = X_train_ex.drop(columns=['day'], axis=1)
test_set_ex_2 = test_set_ex.drop(columns=['day'], axis=1)

In [ ]:
y_pred = AveragingBlending(models, X_train_ex_2, y_train, test_set_ex_2)
print(len(y_pred))
y_pred

In [ ]:
y_pred.shape

In [ ]:
for_submission = pd.read_csv("sample_submission.csv.zip")

In [ ]:
for_submission['visitors'] = y_pred

In [ ]:
for_submission.head()

In [ ]:
for_submission.to_csv('submission_ensemble_gxl_ex_2.csv', index=False)

In [ ]:
from google.colab import files
files.download("submission_ensemble_gxl_ex_2.csv")

##### 새로운 앙상블 모델

In [ ]:
gboost = GradientBoostingRegressor(random_state=2019)
xgboost = xgb.XGBRegressor(random_state=2019, n_estimators=80, learning_rate=0.05, max_depth=7)
lightgbm = lgb.LGBMRegressor(random_state=2019,  n_estimators=80, learning_rate=0.05, num_leaves=60)
knn = KNeighborsRegressor(n_jobs=-1)

models_2 = [{'model':gboost, 'name':'GradientBoosting'}, {'model':xgboost, 'name':'XGBoost'},
            {'model':lightgbm, 'name':'LightGBM'}, {'model':knn, 'name':'KNN'}]

In [ ]:
y_pred = AveragingBlending(models_2, X_train_ex_2, y_train, test_set_ex_2)
print(len(y_pred))
y_pred

In [ ]:
y_pred.shape

In [ ]:
for_submission = pd.read_csv("sample_submission.csv.zip")

In [ ]:
for_submission['visitors'] = y_pred

In [ ]:
for_submission.head()

In [ ]:
for_submission.to_csv('submission_ensemble_gxlk_ex_2.csv', index=False)

In [ ]:
from google.colab import files
files.download("submission_ensemble_gxlk_ex_2.csv")